Concurrency (동시성)

    - CPU 가용성을 극대화하기 위해 Parrallelism의 단점 및 어려움을 구현 레벨에서 해결하기 위한 방법
    - 동시 작업에서 일정양 처리 후에 다음 작업으로 넘기는 방식
    - 제어권을 주고 받는 작업 처리 패턴, 병렬적은 아니지만 유사한 처리 방식이다.

Concurrency VS Parrallelism (병렬성)

- 동시성: 논리적, 동시 실행 패턴, 싱글코어 및 멀티코어에서 실행가능, 한 개의 작업 공유 처리, 디버깅이 매우 어려움, mutex, deadlock 
- 병렬성: 물리적으로 동실 실행, 멀티코에서 구현 가능함, 주로 별개의 작업 처리, 디버깅 어려움, openMD, MPI, CUDA



Blocking IO vs Non-blocking IO
<br><br>
Blocking IO

    - system call -> kernel IO 작업 완료까지 응답을 대기함
    - 제어권이 IO작업에 있다. -> kernel이 소유함 -> response 전까지 대기 (block) -> 다른 작업을 수행할 수 없다.
    
Non-blocking IO

    - system call -> kernel이 IO 작업 완료에 무관하게 즉시 응답한다.
    - 제어권이 IO 작업에 있다 -> user process -> 다른 작업을 수행할 수 있다. -> 주기적으로 시스템 콜을 통해서 IO 작업 완료 여부를 확인한다.
    
Async vs Sync

    Async: IO 작업 완료 여부에 대한 Noty는 호출되는 함수 (kernel)에서 호출하는 함수 (user process)로 간다.
    Sync: IO 작업 완료 여부에 대한 Noty는 호출하는 함수 (user process)에서 호출되는 함수 (kernel)로 간다.

    


![nn](./IO1.jpg) <br>
![nn](./IO2.jpg) <br>
![nn](./IO3.png) <br>
reference: https://limdongjin.github.io/concepts/blocking-non-blocking-io.html

CPU bound vs I/O bound

    CPU bound

    - process 진행이 CPU 속도에 의해 제한된다. -> 행렬곱, 고속연산, 파일 압축, 집합 연산 등등
    - CPU연산 위주 작업

    I/O bound
    
    - 파일쓰기, 디스크 작업, 네트워크 통신, 시리얼 포트 송수신 -> 작업에 의해서 병목에 의해 시간이 결정된다.
    - CPU 성능이 수행시간 단축으로 크게 영향을 끼치지 않는다.

Memory binding, cache bounding <br>
작업 목적에 따라서 적절한 동시성 라이브러리 선택이 중요하다.  
최종비교 

    - Multiprocessing: Multiple process, 고가용성 CPU Utilization -> CPU-Bound -> 10개의 부엌에 10명의 요리사가 있고 10개의 요리를 한다. 각각 자원끼리 작업끼리 독립적이다.

    - threading: In Single(Multi) process and multiple threads, OS decides task switching -> Fast I/O bound -> 1개의 부엌, 10명의 요리사, 10개의 요리
    
    - AsyncIO: In Single process and single thread and cooperative multitasking, tasks cooperatively decide switching -> Slow I/O-Bound -> 1개의 부엌, 1명 요리사가 10개의 요리를 한다.

Synchronous, I/O bound, requests package



In [10]:
# requests는 서버에서 정보를 가져오는 패키지이다.
# !pip install requests
import requests
import time

# 실행함수1 다운로드
def request_site(url, session):
    # session 확인
    print(session)
    print(session.headers)
    """
    SSLError: HTTPSConnectionPool
    신뢰할 수 있는 SSL 인증서로 인해서 발생한다.
    get에 verify를 false로 바꾼다.
    
    또는 

    openssl, urllib3 등등의 버전이 맞지 않기 때문일 수 있다.

    py file로도 안 되는 것을 보면 ai 가상환경을 다시 버전에 맞게 만들어 봐야겠다.
    """
    with session.get(url, verify=False) as response:
        print(f'[Read contents: {len(response.content)}, Status Code: {response.status_code}] from {url}')


# 실행함수2 요청
def request_all_sites(urls):
    # 서버에 접속한 시점에서 웹브라우저에서 정보를 가져옴
    with requests.Session() as session:
        for url in urls:

            request_site(url, session)

  


# sync blocking IO, 순서대로 방문한다.
def main():
    # test URLs
    urls = [
        'https://www.jython.org',
        'http://olympus.realpython.org/dice', # https로 쓰면 SSL 오류 나더라 ㅋㅋㅋㅋ
        'https://realpython.com',
    ] *3

    # 실행시간 측정
    start_time = time.time()

    # 실행
    request_all_sites(urls)

    # 실행 시간 종료
    duration = time.time() - start_time

    # 결과 출력
    print()
    print(f'Downloaded {len(urls)} sites in {duration} seconds')



# 우리가 생각 없이 코드를 작성하면 작성자에 의해서 sync blocking IO이다.
# 단지, 패키지나 low level에서 다르게 작성한다면 그거에 맞춰서 움직일 것이다.

In [11]:
if __name__ == '__main__':
    main()

{'User-Agent': 'python-requests/2.25.1', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'Connection': 'keep-alive'}


c:\Users\rokie\anaconda3\envs\ai\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.jython.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


[Read contents: 10782, Status Code: 200] from https://www.jython.org
{'User-Agent': 'python-requests/2.25.1', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'Connection': 'keep-alive'}
[Read contents: 275, Status Code: 200] from http://olympus.realpython.org/dice
{'User-Agent': 'python-requests/2.25.1', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'Connection': 'keep-alive'}


c:\Users\rokie\anaconda3\envs\ai\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'realpython.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


[Read contents: 57145, Status Code: 200] from https://realpython.com
{'User-Agent': 'python-requests/2.25.1', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'Connection': 'keep-alive'}
[Read contents: 10782, Status Code: 200] from https://www.jython.org
{'User-Agent': 'python-requests/2.25.1', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'Connection': 'keep-alive'}
[Read contents: 275, Status Code: 200] from http://olympus.realpython.org/dice
{'User-Agent': 'python-requests/2.25.1', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'Connection': 'keep-alive'}


...

[Read contents: 57146, Status Code: 200] from https://realpython.com
{'User-Agent': 'python-requests/2.25.1', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'Connection': 'keep-alive'}
[Read contents: 10782, Status Code: 200] from https://www.jython.org
{'User-Agent': 'python-requests/2.25.1', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'Connection': 'keep-alive'}


c:\Users\rokie\anaconda3\envs\ai\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.jython.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


[Read contents: 275, Status Code: 200] from http://olympus.realpython.org/dice
{'User-Agent': 'python-requests/2.25.1', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'Connection': 'keep-alive'}


c:\Users\rokie\anaconda3\envs\ai\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'realpython.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


[Read contents: 57146, Status Code: 200] from https://realpython.com

Downloaded 9 sites in 3.4102303981781006 seconds
